### 1~3 각 개별 jupyter note에서 전처리 후, pickle 또는 csv저장.

1. 전국일방통행도로표준데이터
    * 시도명, 지정사유, 지정연도, 도로폭, 도로차로수, 보차분리여부\
    
2. 전국어린이보호구역표준데이터 시도명 작업 필요
    * 시설종류, 관할경찰서명, CCTV설치여부, CCTV설치대수, 제공기관명, 보호구역도로폭

3. 전국노인장애인보호구역표준데이터
    * 시도명, 제한속도, CCTV설치여부, CCTV설치대수, 보호구역도로폭

In [142]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

pd_data = pd.read_csv('전국노인장애인보호구역표준데이터.csv', encoding='cp949')
# EDA : Exploratory Data Aanalysis
print(pd_data.columns)



Index(['장소유형코드', '대상시설명', '시도명', '시군구명', '시군구코드', '소재지도로명주소', '소재지지번주소', '위도',
       '경도', '제한속도', '관리기관명', '관리기관전화번호', '관할경찰서명', 'CCTV설치여부', 'CCTV설치대수',
       '보호구역도로폭', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')


In [143]:
column_selection = ['시도명', '제한속도', 'CCTV설치여부', 'CCTV설치대수', '보호구역도로폭']
pd_data = pd_data[column_selection]

print(pd_data.columns)
print(pd_data.dtypes)

Index(['시도명', '제한속도', 'CCTV설치여부', 'CCTV설치대수', '보호구역도로폭'], dtype='object')
시도명          object
제한속도          int64
CCTV설치여부     object
CCTV설치대수    float64
보호구역도로폭      object
dtype: object


In [144]:
# 시도명
# 결측치 없음
# 이상치 없음, 하지만 데이터 통합분석이 목적이므로 
# 전북과 전남 -> 전라도 경북과 경남 -> 경상도, 충북과 충남 -> 충청도로 변경

print(pd_data['시도명'].unique())
print(pd_data['시도명'].hasnans)

filter1 = pd_data['시도명']=='전라북도'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter1, '시도명'] = '전라도'

filter2 = pd_data['시도명']=='전라남도'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter2, '시도명'] = '전라도'

filter3 = pd_data['시도명']=='경상북도'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter3, '시도명'] = '경상도'

filter4 = pd_data['시도명']=='경상남도'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter4, '시도명'] = '경상도'

filter5 = pd_data['시도명']=='충청북도'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter3, '시도명'] = '충청도'

filter6 = pd_data['시도명']=='충청남도'
# pd_data[filter1]['시도명'] = '강원도' #warning
pd_data.loc[filter4, '시도명'] = '충청도'

print(pd_data['시도명'].unique())
print(pd_data['시도명'].hasnans)

['서울특별시' '경상북도' '충청남도' '강원도' '경기도' '전라북도' '대전광역시' '경상남도' '인천광역시' '울산광역시'
 '부산광역시' '대구광역시' '충청북도' '전라남도' '광주광역시' '세종특별자치시' '제주특별자치도']
False
['서울특별시' '충청도' '충청남도' '강원도' '경기도' '전라도' '대전광역시' '인천광역시' '울산광역시' '부산광역시'
 '대구광역시' '충청북도' '광주광역시' '세종특별자치시' '제주특별자치도']
False


In [145]:
# 제한속도
# 결측치 없음
# 이상치 : 20부터60까지 다양한 값을 가지나 10단위 범위로 지정(1의자리 버림)하여 카운트 용이하게 변경(20 ~ 60)



print(pd_data['제한속도'].unique())
print(pd_data['제한속도'].hasnans)


for i in pd_data['제한속도']:
    pd_data['제한속도'].replace(i, (i//10)*10, inplace=True)

        
print(pd_data['제한속도'].unique())
print(pd_data['제한속도'].hasnans)
print(pd_data['제한속도'].value_counts()) 

[30 50 60 40 38 39 41 42 43 44 45 46 47 48 49 51 52 53 54 55 31 32 33 34
 35 36 37 20]
False
[30 50 60 40 20]
False
30    2374
50      91
40      34
60      17
20       2
Name: 제한속도, dtype: int64


In [146]:
# CCTV설치여부
# 처리 불필요
print(pd_data['CCTV설치여부'].unique())
print(pd_data['CCTV설치여부'].hasnans)

['Y' 'N']
False


In [147]:
# CCTV설치대수
# 결측치(NaN) = '알 수 없음'으로 변경
print(pd_data['CCTV설치대수'].unique())
print(pd_data['CCTV설치대수'].hasnans)
print(pd_data['CCTV설치대수'].value_counts(dropna=False))

pd_data['CCTV설치대수'].replace(np.NaN, '알 수 없음', inplace=True)

print(pd_data['CCTV설치대수'].unique())
print(pd_data['CCTV설치대수'].hasnans)
print(pd_data['CCTV설치대수'].value_counts(dropna=False))

[ 1. nan  0.  2.  4.  3.  5.  8.  9.]
True
NaN    1551
0.0     568
1.0     273
2.0      81
3.0      20
4.0      14
5.0       9
8.0       1
9.0       1
Name: CCTV설치대수, dtype: int64
[1.0 '알 수 없음' 0.0 2.0 4.0 3.0 5.0 8.0 9.0]
False
알 수 없음    1551
0.0        568
1.0        273
2.0         81
3.0         20
4.0         14
5.0          9
8.0          1
9.0          1
Name: CCTV설치대수, dtype: int64


In [148]:
# 보호구역도로폭

print(pd_data['보호구역도로폭'].unique())
print(pd_data['보호구역도로폭'].hasnans)
print(pd_data['보호구역도로폭'].value_counts())


['3' '6' '7' '8' '9' nan '7.5' '6.5' '15' '11' '4' '20' '12' '13' '28'
 '20~34' '10' '15~20' '24' '25' '5' '14' '6~12' '16' '17~22' '4.5~5' '4.5'
 '20~23' '6~8' '33' '30' '12~13' '10~12' '8~9' '9~10' '4~30' '7~12' '35'
 '9.6~14.6' '14~25' '12~20' '21' '5~20' '1' '6~14' '5~7' '5~6' '7~11'
 '8~20' '4~8' '4~7' '18' '16~20' '5.5' '23' '6~10' '10~30' '6~7' '8~10'
 '36' '6~25' '22' '5.2' '10~15' '8.5' '6.5~12' '5~10' '0' '29' '5~8' '5~9'
 '33~35' '8 ~25' '7~9' '13~14' '5~11' '5~30' '5~15' '10~14' '26' '7~8'
 '4.5~6.5' '18~36' '5~16' '17' '10~35' '19' '8~12' '6~9' '7.5~12' '4~8.5'
 '5.0~11' '7.5~8' '8.5~25' '8.5~11' '6~13' '7~10' '40' '8~14' '20~30'
 '3~5' '8~35' '12~16' '13~15' '7~15' '5.7' '8.4' '3~4' '6.3~14.8' '2']
True
6          301
8          281
7          189
7~11        85
10          68
          ... 
4~30         1
4.5~6.5      1
18~36        1
5~16         1
5~20         1
Name: 보호구역도로폭, Length: 109, dtype: int64


In [149]:
# 보호구역도로폭
# 오류치: 숫자~숫자 형태의 문자열 --> '숫자a'와 '숫자b'의 평균치(np.float64) 값으로 대체
# 결측치: 전체 도로폭 평균치로 대체.
# 전체 값 소수점 1째 자리까지 반올림(평균치가 10.005709이 나와 자리수 통일)

        
def myfn1(x):
    if type(x) == type(' '):
        if '~' in x:
            m = np.array(x.split('~')).astype(np.float64).mean()
            return str(m)
    
  
# 도로폭: 오류치 처리
# 숫자~숫자 형태의 문자열 --> '숫자a'와 '숫자b'의 평균치(np.float64) 값으로 대체
#print(pd_data['보호구역도로폭'].value_counts())
y = pd_data['보호구역도로폭'].apply(myfn1)
print(y.value_counts())



# 도로폭: 결측치 처리
# 전체 도로폭 평균치로 대체.
#print(pd_data['보호구역도로폭'].isna().value_counts())
y = y.astype(np.float64)
y = y.replace(np.NaN, y.mean())
pd_data['보호구역도로폭'] = y

for i in pd_data['보호구역도로폭']:
    pd_data['보호구역도로폭'].replace(i, round(i, 1), inplace=True)

# print(pd_data['보호구역도로폭'].unique())
# print(pd_data['보호구역도로폭'].hasnans)
print(pd_data['보호구역도로폭'].value_counts(dropna=False))

9.0      93
7.0      44
12.5     20
18.0      8
6.0       8
6.5       6
5.5       6
10.0      6
8.0       6
17.5      5
7.5       5
11.0      5
14.0      4
9.5       3
21.5      3
8.5       3
15.5      2
9.75      2
4.0       2
27.0      2
10.55     2
19.5      2
22.5      1
3.5       1
25.0      1
16.75     1
7.75      1
6.25      1
4.75      1
20.0      1
16.0      1
12.0      1
13.5      1
16.5      1
34.0      1
17.0      1
9.25      1
12.1      1
10.5      1
Name: 보호구역도로폭, dtype: int64
10.0    2270
9.0       93
7.0       44
12.5      20
18.0       8
6.0        8
6.5        6
5.5        6
8.0        6
11.0       5
17.5       5
7.5        5
14.0       4
8.5        3
21.5       3
9.5        3
10.6       2
9.8        2
4.0        2
19.5       2
27.0       2
15.5       2
6.2        1
7.8        1
25.0       1
16.8       1
10.5       1
3.5        1
22.5       1
17.0       1
12.0       1
13.5       1
16.5       1
12.1       1
9.2        1
20.0       1
4.8        1
16.0       1
34.0      

In [52]:
print(pd_data.shape)
print(pd_data.describe(include='all'))

for e in pd_data:
    print(e, pd_data[e].hasnans)
    if pd_data[e].hasnans==True:
        print(pd_data[e].isna().value_counts())

(2518, 5)
         시도명         제한속도 CCTV설치여부    CCTV설치대수 보호구역도로폭
count   2518  2518.000000     2518  967.000000    1484
unique    15          NaN        2         NaN     109
top     충청남도          NaN        N         NaN       6
freq     606          NaN     2010         NaN     301
mean     NaN    31.052423      NaN    0.633919     NaN
std      NaN     4.567556      NaN    1.008457     NaN
min      NaN    20.000000      NaN    0.000000     NaN
25%      NaN    30.000000      NaN    0.000000     NaN
50%      NaN    30.000000      NaN    0.000000     NaN
75%      NaN    30.000000      NaN    1.000000     NaN
max      NaN    60.000000      NaN    9.000000     NaN
시도명 False
제한속도 False
CCTV설치여부 False
CCTV설치대수 True
True     1551
False     967
Name: CCTV설치대수, dtype: int64
보호구역도로폭 True
False    1484
True     1034
Name: 보호구역도로폭, dtype: int64
